## External tcode

In [1]:
import pandas as pd
import random
from prepare_data import preprocess_data_czech
from eval import comapre_unidist_cont, compute_ngram_metrics, comapre_unidist_cat, compute_2d_categorical_metrics
import datetime
import calendar
import numpy as np
pd.set_option('display.max_rows', 80)

In [2]:
from myctgan import CTGAN
from data_transformer import DataTransformer
from data_sampler import DataSampler
#from train import Train
import numpy as np
import tensorflow as tf
import pandas as pd
from prepare_data import preprocess_data_czech
import random

2023-10-11 22:11:52.349448: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [3]:
df = pd.read_csv('../DATA/tr_by_acct_w_age.csv')
raw_data = preprocess_data_czech(df)
raw = raw_data[['account_id', 'type', 'amount', 'tcode', 'datetime', 'year', 'month', 'dow', 'day', 'dtme','td']]
df = raw.copy()
final_raw = df[['amount', 'tcode', 'td', 'day', 'dow', 'dtme', 'month']]

In [4]:
synth_bf = pd.read_csv('../DATA/banksformer.csv')

## visualize

In [5]:
from sdmetrics.reports import utils

In [6]:
synth_bf = pd.read_csv('../DATA/banksformer.csv')
synth_bf = synth_bf[['amount', 'date', 'tcode', 'old_account_id','days_passed']]
synth_bf = synth_bf.rename(columns={'old_account_id': 'account_id', 'days_passed': 'td'})
# Convert the 'date' column to datetime type
synth_bf['date'] = pd.to_datetime(synth_bf['date'])

# Extract year, month, and day into separate columns
synth_bf['year'] = synth_bf['date'].dt.year
synth_bf['month'] = synth_bf['date'].dt.month
synth_bf['day'] = synth_bf['date'].dt.day

synth_bf['type'] = synth_bf['tcode'].str.split('__').str[0]
synth_bf['raw_amount'] = synth_bf.apply(lambda row: row['amount'] if row['type'] == 'CREDIT' else -row['amount'], axis=1)

synthbf_sorted = synth_bf.sort_values(['account_id', 'year', 'month', 'day'])
synthbf_cf = synth_bf[["account_id", "month", "raw_amount", "year"]].groupby(["account_id", "month", "year"],as_index=False)["raw_amount"].sum()
synth_bf['td_new'] = synth_bf['td']
synth_bf = synth_bf[['amount', 'tcode', 'account_id', 'td_new', 'date','month', 'day', 'type', 'raw_amount']]


In [7]:
real = pd.read_csv('../DATA/tr_by_acct_w_age.csv')
real = real.sort_values(by = ["account_id", "date"])
real = preprocess_data_czech(real)
real_cf = real[["account_id", "month", "raw_amount", "year"]].groupby(["account_id", "month", "year"],as_index=False)["raw_amount"].sum()
real_sorted = real.sort_values(['account_id', 'year', 'month', 'day'])
real['td_new'] = real['td']

In [8]:
synth = pd.read_csv('../DATA/synth_exp_B21.csv')
# Convert the 'date' column to datetime type
synth['date'] = pd.to_datetime(synth['date'])

# Extract year, month, and day into separate columns
synth['year'] = synth['date'].dt.year
synth['month'] = synth['date'].dt.month
synth['day'] = synth['date'].dt.day

synth['type'] = synth['tcode'].str.split('__').str[0]
synth['raw_amount'] = synth.apply(lambda row: row['amount'] if row['type'] == 'CREDIT' else -row['amount'], axis=1)

synth_sorted = synth.sort_values(['account_id', 'year', 'month', 'day'])

synth_cf = synth[["account_id", "month", "raw_amount", "year"]].groupby(["account_id", "month", "year"],as_index=False)["raw_amount"].sum()

synth["td_new"] = synth[["account_id", "date"]].groupby("account_id").diff()
synth["td_new"] = synth["td_new"].apply(lambda x: x.days)
synth["td_new"].fillna(0.0, inplace=True)
synth = synth[['amount', 'tcode', 'account_id', 'td_new', 'date','month', 'day', 'type', 'raw_amount']]


In [9]:
real_vis = real[['amount', 'tcode', 'account_id', 'td_new','datetime','month', 'day', 'type', 'raw_amount']]
real_vis = real_vis.rename(columns={'datetime':'date'})
real_vis

,amount,tcode,account_id,td_new,date,month,day,type,raw_amount
0,1000.0,CREDIT__CREDIT IN CASH__nan,1,0.0,1995-03-24,3,24,CREDIT,1000.0
1,3679.0,CREDIT__COLLECTION FROM ANOTHER BANK__nan,1,20.0,1995-04-13,4,13,CREDIT,3679.0
2,12600.0,CREDIT__CREDIT IN CASH__nan,1,10.0,1995-04-23,4,23,CREDIT,12600.0
3,19.2,CREDIT__nan__INTEREST CREDITED,1,7.0,1995-04-30,4,30,CREDIT,19.2
4,3679.0,CREDIT__COLLECTION FROM ANOTHER BANK__nan,1,13.0,1995-05-13,5,13,CREDIT,3679.0
...,...,...,...,...,...,...,...,...,...
1056315,25600.0,DEBIT__CASH WITHDRAWAL__nan,11382,2.0,1998-12-02,12,2,DEBIT,-25600.0
1056316,46248.0,CREDIT__COLLECTION FROM ANOTHER BANK__nan,11382,8.0,1998-12-10,12,10,CREDIT,46248.0
1056317,6300.0,DEBIT__CASH WITHDRAWAL__nan,11382,15.0,1998-12-25,12,25,DEBIT,-6300.0
1056318,311.3,CREDIT__nan__INTEREST CREDITED,11382,6.0,1998-12-31,12,31,CREDIT,311.3


In [10]:
dic_metadata = {
    "primary_key": "account_id",
    "columns": {
        "account_id": {
            "sdtype": "id",
           
        },
        "amount": {
            "sdtype": "numerical"
        },
        "raw_amount": {
            "sdtype": "numerical"
        },
        "td_new": {
            "sdtype": "numerical"
        },
       
        "tcode": {
            "sdtype": "categorical"
        },

        "type": {
            "sdtype": "categorical"
        },
        "day": {
            "sdtype": "categorical"
        },
        "month": {
            "sdtype": "categorical"
        },
        "date": {
            "sdtype": "datetime",
            "datetime_format": "%Y-%m-%d"
        }
    }
}


In [11]:
dic_metadata_cf = {
    "primary_key": "account_id",
    "columns": {
        "account_id": {
            "sdtype": "id",
           
        },
        "raw_amount": {
            "sdtype": "numerical"
        },
       
        "month": {
            "sdtype": "categorical"
        },
    }
}

In [13]:
from sdmetrics.reports import utils
from plotly.subplots import make_subplots
import plotly.graph_objs as go

fig1 = utils.get_column_plot(
    real_data=real_cf,
    synthetic_data=synthbf_cf,
    column_name='raw_amount',
    metadata=dic_metadata_cf
)
fig2 = utils.get_column_plot(
    real_data=real_cf,
    synthetic_data=synth_cf,
    column_name = 'raw_amount',
    metadata=dic_metadata_cf
)

# Create subplot with 1 row and 2 columns
subplot_fig = make_subplots(rows=1, cols=2)

# Update subplot with all data traces from fig1 and fig2
for trace in fig1.data:
    subplot_fig.add_trace(trace, row=1, col=1)

for trace in fig2.data:
    trace.showlegend = False
    subplot_fig.add_trace(trace, row=1, col=2)

# subplot_fig.update_yaxes(range=[0, 0.14], row=1, col=1)
# subplot_fig.update_yaxes(range=[0, 0.14], row=1, col=2)

# Set individual subplot titles
subplot_fig.update_layout(
annotations=[
        dict(text="Cash Flow Distribution(Banksformer)", x=0.15, y=1.1, showarrow=False, xref="paper", yref="paper", font=dict(size=14, color="black")),
        dict(text="Cash Flow Distribution(Exp_B21)", x=0.85, y=1.1, showarrow=False, xref="paper", yref="paper", font=dict(size=14, color="black"))
    ]
)